In [1]:
%pip install ../../

Processing /home/hyang/SpuCo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
ERROR: Package 'spuco' requires a different Python: 3.8.5 not in '>=3.10.0'
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

ModuleNotFoundError: No module named 'spuco'

In [18]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [19]:
from spuco.models import model_factory 
from spuco.utils import Trainer
from torch.optim import SGD

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

trainer.train(1)

Epoch 0: 100%|██████████| 938/938 [00:03<00:00, 257.54batch/s, accuracy=96.875%, loss=0.207]  


In [22]:
from spuco.evaluate import Evaluator 

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 100.0
Group (0, 1) Test Accuracy: 0.0
Group (0, 2) Test Accuracy: 0.0
Group (0, 3) Test Accuracy: 0.0
Group (0, 4) Test Accuracy: 0.0
Group (1, 0) Test Accuracy: 0.0
Group (1, 1) Test Accuracy: 100.0
Group (1, 2) Test Accuracy: 0.0
Group (1, 3) Test Accuracy: 0.0
Group (1, 4) Test Accuracy: 0.0
Group (2, 0) Test Accuracy: 0.0
Group (2, 1) Test Accuracy: 0.0
Group (2, 2) Test Accuracy: 100.0
Group (2, 3) Test Accuracy: 0.0
Group (2, 4) Test Accuracy: 0.0
Group (3, 0) Test Accuracy: 0.0
Group (3, 1) Test Accuracy: 0.0
Group (3, 2) Test Accuracy: 0.0
Group (3, 3) Test Accuracy: 100.0
Group (3, 4) Test Accuracy: 0.0
Group (4, 0) Test Accuracy: 0.0
Group (4, 1) Test Accuracy: 0.0
Group (4, 2) Test Accuracy: 0.0
Group (4, 3) Test Accuracy: 0.0
Group (4, 4) Test Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [24]:
from spuco.group_inference import Cluster, ClusterAlg

logits = trainer.get_trainset_outputs()
cluster = Cluster(
    Z=logits,
    class_labels=trainset.labels,
    cluster_alg=ClusterAlg.KMEANS,
    num_clusters=2,
    device=device,
    verbose=True
)
group_partition = cluster.infer_groups()

Clustering class-wise: 100%|██████████| 5/5 [00:13<00:00,  2.68s/it]


In [25]:
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

(0, 0) 12598
(0, 1) 67
(1, 0) 12055
(1, 1) 34
(2, 0) 11218
(2, 1) 45
(3, 0) 12134
(3, 1) 49
(4, 0) 11757
(4, 1) 43


In [26]:
from spuco.evaluate import Evaluator 

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 99.99206223210034
Group (0, 1) Test Accuracy: 0.0
Group (1, 0) Test Accuracy: 99.8257984238905
Group (1, 1) Test Accuracy: 0.0
Group (2, 0) Test Accuracy: 99.91085755036548
Group (2, 1) Test Accuracy: 0.0
Group (3, 0) Test Accuracy: 99.92582825119499
Group (3, 1) Test Accuracy: 0.0
Group (4, 0) Test Accuracy: 99.87241643276346
Group (4, 1) Test Accuracy: 0.0


{(0, 0): 99.99206223210034,
 (0, 1): 0.0,
 (1, 0): 99.8257984238905,
 (1, 1): 0.0,
 (2, 0): 99.91085755036548,
 (2, 1): 0.0,
 (3, 0): 99.92582825119499,
 (3, 1): 0.0,
 (4, 0): 99.87241643276346,
 (4, 1): 0.0}

In [33]:
from torch.optim import SGD
from spuco.invariant_train import GroupBalanceBatchERM, ClassBalanceBatchERM
from spuco.models import model_factory 

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
group_balance_erm = GroupBalanceBatchERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_balance_erm.train()

Epoch 4: 100%|██████████| 938/938 [00:03<00:00, 286.79batch/s, accuracy=100.0%, loss=0.0219]  


In [34]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 99.05437352245863
Group (0, 1) Test Accuracy: 60.283687943262414
Group (0, 2) Test Accuracy: 74.94089834515367
Group (0, 3) Test Accuracy: 69.03073286052009
Group (0, 4) Test Accuracy: 78.01418439716312
Group (1, 0) Test Accuracy: 64.05867970660147
Group (1, 1) Test Accuracy: 97.31051344743277
Group (1, 2) Test Accuracy: 66.66666666666667
Group (1, 3) Test Accuracy: 68.38235294117646
Group (1, 4) Test Accuracy: 64.95098039215686
Group (2, 0) Test Accuracy: 74.13333333333334
Group (2, 1) Test Accuracy: 56.0
Group (2, 2) Test Accuracy: 98.66666666666667
Group (2, 3) Test Accuracy: 9.6
Group (2, 4) Test Accuracy: 48.663101604278076
Group (3, 0) Test Accuracy: 80.40201005025126
Group (3, 1) Test Accuracy: 74.3073047858942
Group (3, 2) Test Accuracy: 7.8085642317380355
Group (3, 3) Test Accuracy: 97.98488664987406
Group (3, 4) Test Accuracy: 72.544080604534
Group (4, 0) Test Accuracy: 12.090680100755668
Group (4, 1) Test Accuracy: 74.05541561712846
Group (4, 2) T

{(0, 0): 99.05437352245863,
 (0, 1): 60.283687943262414,
 (0, 2): 74.94089834515367,
 (0, 3): 69.03073286052009,
 (0, 4): 78.01418439716312,
 (1, 0): 64.05867970660147,
 (1, 1): 97.31051344743277,
 (1, 2): 66.66666666666667,
 (1, 3): 68.38235294117646,
 (1, 4): 64.95098039215686,
 (2, 0): 74.13333333333334,
 (2, 1): 56.0,
 (2, 2): 98.66666666666667,
 (2, 3): 9.6,
 (2, 4): 48.663101604278076,
 (3, 0): 80.40201005025126,
 (3, 1): 74.3073047858942,
 (3, 2): 7.8085642317380355,
 (3, 3): 97.98488664987406,
 (3, 4): 72.544080604534,
 (4, 0): 12.090680100755668,
 (4, 1): 74.05541561712846,
 (4, 2): 29.974811083123427,
 (4, 3): 87.87878787878788,
 (4, 4): 98.23232323232324}

In [35]:
evaluator.worst_group_accuracy

((3, 2), 7.8085642317380355)

In [36]:
evaluator.average_accuracy

98.07091522661091

In [37]:
evaluator.evaluate_spurious_task()

41.56